In [1]:
# Import the Pandas library
import pandas as pd

import sys
sys.path.append('../')

from models.utils import process_excel

df = process_excel('../.test_data/dataprocess.xlsx')

# Assuming `df` is the DataFrame with the color data
color_series = df.drop('NAME', axis=1).stack()  # Drop the 'NAME' column and stack to create a single series
color_codes, unique_colors = pd.factorize(color_series)  # Factorize the entire series to get unique codes

# Map the color codes back into the original DataFrame shape
coded_df = pd.DataFrame(color_codes.reshape(df.shape[0], -1), columns=df.columns[1:])
coded_df.insert(0, 'NAME', df['NAME'])  # Insert the 'NAME' column back into the DataFrame

coded_df



,NAME,2024-12-07 00:00:00,2024-12-08 00:00:00,2024-12-09 00:00:00,2024-12-12 00:00:00,2024-12-13 00:00:00,2024-12-14 00:00:00,2024-12-15 00:00:00,2024-12-16 00:00:00,2024-12-19 00:00:00,...,2024-04-02 00:00:00,2024-04-03 00:00:00.1,2024-04-04 00:00:00,2024-04-05 00:00:00.1,2024-04-08 00:00:00,2024-04-09 00:00:00,2024-04-10 00:00:00.1,2024-04-12 00:00:00.1,2024-04-15 00:00:00,2024-04-16 00:00:00
0,GJ1,0,1,2,0,3,1,2,0,0,...,1,4,6,1,4,4,1,6,4,1
1,GJ2,0,2,0,0,1,1,2,0,3,...,1,4,8,8,1,4,4,8,8,1
2,GJ3,3,1,7,1,1,1,1,2,2,...,1,1,8,4,4,8,8,1,6,1
3,GJ4,0,1,2,3,3,3,2,2,2,...,8,1,6,4,6,1,1,4,8,8
4,GJ5,3,3,1,2,0,1,1,0,0,...,1,4,8,4,4,6,1,4,4,8
5,GJ6,0,3,3,2,3,2,3,0,3,...,4,8,7,8,4,6,4,8,4,1
6,GJ7,2,7,0,0,3,3,3,0,0,...,4,6,8,4,4,8,1,6,4,4
7,GJ8,0,1,0,3,3,1,0,2,2,...,6,8,8,4,4,8,4,8,6,6
8,GJ9,2,3,2,0,3,1,0,0,0,...,8,1,8,4,1,4,4,1,6,8


In [21]:
color_code_to_hex_mapping = dict(enumerate(unique_colors))

color_code_to_hex_mapping



{0: '00FFFF',
 1: 'FFFF00',
 2: 'FF9900',
 3: '00FF00',
 4: 'A9D08E',
 5: '9BC2E6',
 6: 'FFC000',
 7: 'D5A6BD',
 8: '8EA9DB',
 9: 'FF0000'}

In [7]:
# Assuming 'data' is a pandas DataFrame containing your data as provided in the original format
# Convert the date columns (excluding the 'NAME' column)
data = coded_df 

# Processing the DataFrame 'data' to have "date", "name", "color_value" columns
data_long = pd.melt(data, id_vars=['NAME'], var_name='date', value_name='color_value')
data_long['date'] = pd.to_datetime(data_long['date'])

# Convert dates to a numerical value, such as the day of the year
data_long['day_of_year'] = data_long['date'].dt.dayofyear
data_long['name_as_number'] = data_long['NAME'].str.extract('(\d+)').astype(int)

data_long

,NAME,date,color_value,day_of_year,name_as_number
0,GJ1,2024-12-07,0,342,1
1,GJ2,2024-12-07,0,342,2
2,GJ3,2024-12-07,3,342,3
3,GJ4,2024-12-07,0,342,4
4,GJ5,2024-12-07,3,342,5
...,...,...,...,...,...
3019,GJ5,2024-04-16,8,107,5
3020,GJ6,2024-04-16,1,107,6
3021,GJ7,2024-04-16,4,107,7
3022,GJ8,2024-04-16,6,107,8


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Prepare the dataset for Linear Regression
# Updated to include 'name_as_number' as an additional feature
X = data_long[['day_of_year', 'name_as_number']].values  # Features
y = data_long['color_value'].values  # Target

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


print(X_train)
print(y_train)

[[349   2]
 [ 41   2]
 [188   2]
 ...
 [161   6]
 [188   8]
 [118   6]]
[8 1 4 ... 6 6 4]


In [17]:
# Create and train the model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Print model information
print('Model coefficients:', regressor.coef_)
print('Model intercept:', regressor.intercept_)

Model coefficients: [-0.00162443  0.06433792]
Model intercept: 4.537006205916549


In [12]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


# Predicting new values (You would add new 'day_of_year' values here to make predictions)
predicted_colors = regressor.predict(X_test)

# Actual vs Predicted colors
mse = mean_squared_error(y_test, predicted_colors)


print(f'Mean Squared Error (MSE): {mse}')


Mean Squared Error (MSE): 5.6669745729607275


In [18]:

predicted_colors =  [round(x) for x in predicted_colors]

# Calculate the number of correct predictions
correct_predictions = sum(y_test == predicted_colors)
# Calculate the total number of predictions
total_predictions = len(predicted_colors)
# Calculate the percentage of correct predictions
accuracy_percentage = (correct_predictions / total_predictions) * 100
# Calculate the percentage of incorrect predictions
incorrect_percentage = 100 - accuracy_percentage

# Print the values
print(f'Percentage of correct predictions: {accuracy_percentage}%')
print(f'Percentage of incorrect predictions: {incorrect_percentage}%')


[1 1 1 8 5 4 1 8 4 1 8 1 4 5 3 4 8 6 8 1 5 4 6 6 7 8 6 6 6 4 4 6 2 4 6 1 0
 8 5 8 6 5 4 6 1 6 7 4 4 5 1 1 4 6 1 1 5 8 1 6 6 1 0 4 6 8 0 8 4 6 1 1 6 6
 3 4 8 7 4 6 6 7 8 4 8 4 4 8 5 3 4 1 2 8 4 4 1 6 1 4 8 8 4 4 6 4 1 4 5 1 4
 6 5 4 5 4 8 6 8 4 5 4 5 7 4 8 1 4 6 1 4 8 1 4 1 1 8 4 4 5 6 4 6 8 4 1 4 4
 4 6 1 6 4 4 6 6 8 1 1 8 8 8 1 1 4 5 1 4 4 5 1 5 1 4 4 6 8 6 8 6 1 4 6 4 6
 8 5 1 1 8 4 3 4 5 5 6 7 6 6 1 1 4 5 4 1 8 4 6 6 7 8 1 4 4 6 4 1 6 4 4 0 4
 6 4 6 4 6 6 8 4 5 8 5 6 4 5 1 1 6 8 5 3 4 1 5 4 6 4 5 8 8 1 7 8 4 1 6 8 8
 4 8 6 4 5 4 4 4 1 0 1 1 4 8 1 6 6 6 4 6 4 1 4 6 5 6 6 4 1 8 8 4 4 6 8 8 8
 4 1 1 4 4 7 4 4 1 7 5 7 6 4 1 8 8 8 1 6 4 3 1 1 8 4 1 6 1 1 4 4 4 7 8 8 4
 1 8 6 3 8 6 8 4 1 4 4 6 4 1 1 4 6 8 8 1 6 7 5 4 8 6 1 6 4 4 4 8 5 1 6 4 6
 6 1 4 6 8 5 1 4 4 6 1 7 8 4 8 4 8 4 1 4 1 0 5 4 8 7 4 1 8 6 1 1 4 5 1 4 6
 5 1 4 1 8 7 1 5 8 2 5 7 1 1 6 8 8 4 7 6 5 6 1 6 7 8 4 1 8 7 4 1 1 4 8 8 6
 4 1 8 6 1 8 5 1 0 1 4 6 6 6 1 1 5 8 1 8 7 1 4 5 7 6 5 4 6 8 1 4 7 8 1 4 4
 6 8 8 6 5 4 5 6 4 1 4 4 

In [ ]:
#make a df frome two inputs



data = {
    'NAME': ['GJ1'],
    'date': [pd.to_datetime('2024-12-07')],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)
